### 뉴스기사 요약해보기 프로젝트

이 노트북에서는 뉴스기사 데이터셋(news_summary_more.csv)을 활용하여 **추상적 요약**과 **추출적 요약**을 수행합니다.

### 프로젝트 목표
- **추상적 요약**: seq2seq 모델과 어텐션 메커니즘을 이용하여 뉴스 기사의 본문(`text`)을 입력받아 헤드라인(`headlines`)을 생성합니다.
- **추출적 요약**: Summa 라이브러리의 `summarize` 함수를 사용하여 기사 본문에서 핵심 문장을 추출합니다.

### 평가 루브릭
1. **텍스트 전처리 단계**: 분석, 정제, 정규화, 불용어 제거, 데이터셋 분리, 인코딩 과정이 체계적으로 진행됨.
2. **텍스트 요약 모델 학습**: 모델 학습 중 train/validation loss 감소 추세 확인 및 생성 요약문 내 핵심 단어 포함 확인.
3. **추출적 요약과 추상적 요약 비교**: 두 요약 결과를 문법 완성도 및 핵심 단어 포함 측면에서 비교 분석.


In [26]:
from importlib.metadata import version
import nltk
import tensorflow
import summa
import pandas as pd
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

## Step 1. 데이터 수집하기

In [2]:
import urllib.request
import pandas as pd

In [35]:
# 데이터셋 다운로드
data = pd.read_csv('C:\\Users\\User\\.kaggle\\news_summary_more.csv', nrows=100000)
print('전체 샘플수 :', len(data))


전체 샘플수 : 98401


In [36]:
data.head()


,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [37]:
data.sample(10)


,headlines,text
95921,"I'll run out of here, jokes Karan on hearing K...","At a recent event, filmmaker Karan Johar jokin..."
51104,Pentagon suggests countering cyber attacks wit...,A policy drafted by the US Department of Defen...
67667,Anti-ISIS 'sheikh of snipers' killed in Iraq,"A veteran fighter known as ""sheikh of snipers""..."
52941,"Rohit Sharma tries trolling Chahal, gets troll...",Indian cricketer Rohit Sharma took to Instagra...
61485,Pay lawyers after approval: Delhi Dy CM tells ...,Delhi Deputy CM Manish Sisodia has directed al...
70915,"40,000 people affected due to flash floods in ...","Nearly 40,000 people in Assam's Sonitpur were ..."
63980,"Man flies 8,000 ft above S Africa using 100 he...","A British adventurer Tom Morgan flew 8,000 fee..."
58429,"Film industry today has become a ""sabzi mandi""...",Veteran actor Dharmendra has said that the fil...
49387,"Ambani's Reliance Industries to invest â¹2,50...",Reliance Industries Chairman Mukesh Ambani has...
76182,"Hockey turf in Rajasthan cut up, being used as...","Pieces of a hockey artificial turf, known as A..."


## Step 2. 데이터 전처리하기 (추상적 요약)

데이터셋은 기사의 본문(`text`)과 헤드라인(`headlines`) 열로 구성되어 있습니다. 
이 단계에서는 중복/Null 제거, 텍스트 정규화, 불용어 제거, 길이 제한, 시작/종료 토큰 추가, 그리고 학습/테스트 데이터 분리를 수행합니다.

In [ ]:
# 1. 각 텍스트별 등장 횟수를 계산하여, 1회보다 많이 등장한 항목을 확인
duplicate_counts = data['text'].value_counts()
duplicates = duplicate_counts[duplicate_counts > 1]
print("중복된 텍스트와 해당 중복 횟수:")
print(duplicates)

# 2. 중복된 행 전체를 출력 (keep=False 옵션으로 모든 중복 행을 반환)
duplicate_rows = data[data.duplicated(subset=['text'], keep=False)]
print("\n중복된 뉴스 기사 전체:")
print(duplicate_rows.sort_values(by='text'))


In [38]:
df=data
# Check for duplicates in each individual column ('headlines' and 'text')
duplicates_headlines = df.duplicated(subset=['headlines']).sum()
duplicates_text = df.duplicated(subset=['text']).sum()

duplicates_headlines, duplicates_text


(121, 41)

In [39]:
print('text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())
print('headlines 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())

text 열에서 중복을 배제한 유일한 샘플의 수 : 98360
headlines 열에서 중복을 배제한 유일한 샘플의 수 : 98280


In [30]:
# 중복 및 Null 값 제거
data.drop_duplicates(subset=['text'], inplace=True)
# data.dropna(inplace=True)
print('Null 값 확인:')
print(data.isnull().sum())


Null 값 확인:
headlines    0
text         0
dtype: int64


In [31]:
print('전체 샘플수 :', (len(data)))

전체 샘플수 : 98360


정규화 사전

In [32]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

print("정규화 사전의 수: ", len(contractions))

정규화 사전의 수:  120


In [33]:
print('불용어 개수 :', len(stopwords.words('english') ))
print(stopwords.words('english'))

불용어 개수 : 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 's

In [9]:
# 데이터 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거 Ex) my husband (and myself!) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub(r"'s\b","", sentence) # 소유격 제거. Ex) roland's -> roland
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub('[m]{2,}', 'mm', sentence) # m이 3개 이상이면 2개로 변경. Ex) ummmmmmm yeah -> umm yeah

    # 불용어 제거 (text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stopwords.words('english') if len(word) > 1)
    # 불용어 미제거 (healines)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens
print('=3')

=3


In [10]:
# 텍스트와 헤드라인 전처리
clean_text = [preprocess_sentence(s) for s in data['text']]
clean_headlines = [preprocess_sentence(s, remove_stopwords=False) for s in data['headlines']]

data['text'] = clean_text
data['headlines'] = clean_headlines

# 빈 문자열을 NaN으로 치환 후 제거
data.replace('', np.nan, inplace=True)
data.dropna(inplace=True)

print('전처리 후 샘플:')
print(data[['text', 'headlines']].head())


C:\Users\User\AppData\Local\Temp\ipykernel_13992\2321396210.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거


전처리 후 샘플:
                                                text  \
0  saurav kant alumnus upgrad iiit pg program mac...   
1  kunal shah credit card bill payment platform c...   
2  new zealand defeated india wickets fourth odi ...   
3  aegon life iterm insurance plan customers enjo...   
4  speaking sexual harassment allegations rajkuma...   

                                           headlines  
0  upgrad learner switches to career in ml al wit...  
1  delhi techie wins free food from swiggy for on...  
2  new zealand end rohit sharma led india match w...  
3  aegon life iterm insurance plan helps customer...  
4  have known hirani for yrs what if metoo claims...  


전처리 후 emptied 된 샘플 확인 후 제거

In [15]:
data.isnull().sum()

headlines         0
text              0
decoder_input     0
decoder_target    0
dtype: int64

In [16]:
data.dropna(axis=0, inplace=True)
print('전체 샘플수 :', (len(data)))

전체 샘플수 : 27105


# 연산 속도 최적화를 위한 전처리

In [ ]:
import pandas as pd
from multiprocessing import Pool, cpu_count

# 병렬 처리를 위한 함수
def parallel_apply(texts, func, remove_stopwords=True):
    with Pool(cpu_count()) as p:
        result = p.starmap(func, [(s, remove_stopwords) for s in texts])
    return result

# 전처리 함수
def preprocess_sentence(sentence, remove_stopwords=True):
    sentence = sentence.lower()
    sentence = BeautifulSoup(sentence, "lxml").text
    sentence = re.sub(r'\([^)]*\)', '', sentence)
    sentence = re.sub('"', '', sentence)
    sentence = re.sub(r"'s\b", "", sentence)
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    sentence = re.sub('[m]{2,}', 'mm', sentence)

    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if word not in stopwords.words('english') if len(word) > 1)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

# 전처리 수행
data['clean_text'] = parallel_apply(data['text'], preprocess_sentence)
data['clean_headlines'] = parallel_apply(data['headlines'], preprocess_sentence, remove_stopwords=False)


# 전처리 과정 전후의 샘플 수의 급격한 차이에 대한 분석

### 전처리 전후 샘플의 유사도 계산

In [ ]:
original_texts = data['text'].sample(1000, random_state=42)  # 원본 텍스트
processed_texts = original_texts.apply(preprocess_sentence)  # 전처리 적용

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
original_tfidf = vectorizer.fit_transform(original_texts)
processed_tfidf = vectorizer.fit_transform(processed_texts)

# 코사인 유사도 계산
original_similarity = cosine_similarity(original_tfidf)
processed_similarity = cosine_similarity(processed_tfidf)

# 유사도 평균 비교
original_similarity_mean = np.mean(original_similarity)
processed_similarity_mean = np.mean(processed_similarity)

original_similarity_mean, processed_similarity_mean

==========================================================================

In [11]:
# 데이터 길이 분포 확인 (필요시 시각화)
text_len = data['text'].apply(lambda x: len(x.split()))
headlines_len = data['headlines'].apply(lambda x: len(x.split()))
print('텍스트 길이 - 최소:', text_len.min(), '최대:', text_len.max(), '평균:', text_len.mean())
print('헤드라인 길이 - 최소:', headlines_len.min(), '최대:', headlines_len.max(), '평균:', headlines_len.mean())

# 최대 길이 제한 (추상적 요약을 위한) 
text_max_len = 50
headlines_max_len = 8
data = data[data['text'].apply(lambda x: len(x.split()) <= text_max_len)]
data = data[data['headlines'].apply(lambda x: len(x.split()) <= headlines_max_len)]
print('최대 길이 조건 후 샘플 수:', len(data))

# 시작 토큰과 종료 토큰 추가
data['decoder_input'] = data['headlines'].apply(lambda x: 'sostoken ' + x)
data['decoder_target'] = data['headlines'].apply(lambda x: x + ' eostoken')

print(data[['headlines', 'decoder_input', 'decoder_target']].head())


텍스트 길이 - 최소: 1 최대: 60 평균: 35.09968483123221
헤드라인 길이 - 최소: 1 최대: 16 평균: 9.299532330215534
최대 길이 조건 후 샘플 수: 27105
                                            headlines  \
19  odisha cm patnaik controls mining mafia union ...   
21  isro unveils bengaluru centre for manned space...   
22              killed injured in saudi arabia floods   
29  seat cushions from missing plane carrying foot...   
36  agustawestland scam accused rajiv saxena extra...   

                                        decoder_input  \
19  sostoken odisha cm patnaik controls mining maf...   
21  sostoken isro unveils bengaluru centre for man...   
22     sostoken killed injured in saudi arabia floods   
29  sostoken seat cushions from missing plane carr...   
36  sostoken agustawestland scam accused rajiv sax...   

                                       decoder_target  
19  odisha cm patnaik controls mining mafia union ...  
21  isro unveils bengaluru centre for manned space...  
22     killed injured in saudi ara

In [12]:
# 데이터 분리 및 셔플
import numpy as np

encoder_input = np.array(data['text'])
decoder_input = np.array(data['decoder_input'])
decoder_target = np.array(data['decoder_target'])

indices = np.arange(len(encoder_input))
np.random.shuffle(indices)
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

n_val = int(len(encoder_input) * 0.2)
encoder_input_train = encoder_input[:-n_val]
decoder_input_train = decoder_input[:-n_val]
decoder_target_train = decoder_target[:-n_val]
encoder_input_test = encoder_input[-n_val:]
decoder_input_test = decoder_input[-n_val:]
decoder_target_test = decoder_target[-n_val:]

print('훈련 샘플 수:', len(encoder_input_train))
print('테스트 샘플 수:', len(encoder_input_test))


훈련 샘플 수: 21684
테스트 샘플 수: 5421


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 인코더 텍스트 정수 인코딩
src_vocab = 8000
src_tokenizer = Tokenizer(num_words=src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)
encoder_input_train_seq = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test_seq = src_tokenizer.texts_to_sequences(encoder_input_test)

encoder_input_train_seq = pad_sequences(encoder_input_train_seq, maxlen=text_max_len, padding='post')
encoder_input_test_seq = pad_sequences(encoder_input_test_seq, maxlen=text_max_len, padding='post')

# 디코더 텍스트 정수 인코딩
tar_vocab = 2000
tar_tokenizer = Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

decoder_input_train_seq = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_input_test_seq = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_train_seq = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_target_test_seq = tar_tokenizer.texts_to_sequences(decoder_target_test)

decoder_input_train_seq = pad_sequences(decoder_input_train_seq, maxlen=headlines_max_len+1, padding='post')
decoder_input_test_seq = pad_sequences(decoder_input_test_seq, maxlen=headlines_max_len+1, padding='post')
decoder_target_train_seq = pad_sequences(decoder_target_train_seq, maxlen=headlines_max_len+1, padding='post')
decoder_target_test_seq = pad_sequences(decoder_target_test_seq, maxlen=headlines_max_len+1, padding='post')


=================================================================================

## train data dictionary

원문 encoder_input_train 단어집합

In [ ]:
src_tokenizer = Tokenizer() # 토크나이저 정의
src_tokenizer.fit_on_texts(encoder_input_train) # 입력된 데이터로부터 단어 집합 생성
print('=3')

In [ ]:
threshold = 5
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
src_vocab = 8000
src_tokenizer = Tokenizer(num_words=src_vocab) # 단어 집합의 크기를 8,000으로 제한
src_tokenizer.fit_on_texts(encoder_input_train) # 단어 집합 재생성
print('=3')

In [ ]:
# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

# 잘 진행되었는지 샘플 출력
print(encoder_input_train[:3])

## target data dictionary

In [ ]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)
print('=3')

In [ ]:
threshold = 3
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
tar_vocab = 2000
tar_tokenizer = Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

# 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)

# 잘 변환되었는지 확인
print('input')
print('input ',decoder_input_train[:5])
print('target')
print('decoder ',decoder_target_train[:5])

In [ ]:
drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1]

print('삭제할 훈련 데이터의 개수 :', len(drop_train))
print('삭제할 테스트 데이터의 개수 :', len(drop_test))

encoder_input_train = [sentence for index, sentence in enumerate(encoder_input_train) if index not in drop_train]
decoder_input_train = [sentence for index, sentence in enumerate(decoder_input_train) if index not in drop_train]
decoder_target_train = [sentence for index, sentence in enumerate(decoder_target_train) if index not in drop_train]

encoder_input_test = [sentence for index, sentence in enumerate(encoder_input_test) if index not in drop_test]
decoder_input_test = [sentence for index, sentence in enumerate(decoder_input_test) if index not in drop_test]
decoder_target_test = [sentence for index, sentence in enumerate(decoder_target_test) if index not in drop_test]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

In [ ]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen=text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen=text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen=summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen=summary_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen=summary_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen=summary_max_len, padding='post')
print('=3')

## Step 3. 어텐션 메커니즘을 사용한 seq2seq 모델 구성 (추상적 요약)

인코더-디코더 구조에 어텐션 메커니즘을 추가한 seq2seq 모델을 구성합니다.

In [14]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate, Attention
from tensorflow.keras.models import Model

latent_dim = 256

# 인코더
encoder_inputs = Input(shape=(text_max_len,))
enc_emb = Embedding(src_vocab, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# 디코더
decoder_inputs = Input(shape=(headlines_max_len+1,))
dec_emb = Embedding(tar_vocab, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# 어텐션 레이어
attn_layer = Attention()
attn_out = attn_layer([decoder_outputs, encoder_outputs])
decoder_concat = Concatenate(axis=-1)([decoder_outputs, attn_out])

# 출력 레이어
decoder_dense = Dense(tar_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
# 모델 학습 (에포크 수는 상황에 맞게 조정)
history = model.fit([encoder_input_train_seq, decoder_input_train_seq], 
                    decoder_target_train_seq[..., np.newaxis], 
                    batch_size=64, epochs=5, validation_split=0.2)


## Step 4. 실제 결과와 요약문 비교하기 (추상적 요약)

학습된 모델을 이용해 테스트 데이터의 일부에 대해 요약을 생성하고, 실제 헤드라인과 비교합니다.

In [ ]:
# 예시: 테스트 데이터의 첫 번째 샘플에 대해 예측 수행
preds = model.predict([encoder_input_test_seq, decoder_input_test_seq])
predicted_seq = preds.argmax(axis=-1)

# 토크나이저의 인덱스를 단어로 매핑하기 위한 역 딕셔너리 생성
reverse_tar_word_index = {v: k for k, v in tar_tokenizer.word_index.items()}

def decode_sequence(seq):
    return ' '.join([reverse_tar_word_index.get(i, '') for i in seq if i != 0])

print('실제 헤드라인:', decoder_target_test[0])
print('생성된 요약:', decode_sequence(predicted_seq[0]))


## Step 5. Summa를 이용한 추출적 요약

Summa 라이브러리의 `summarize` 함수를 사용하여, 기사 본문(`text`)에서 핵심 내용을 추출합니다.

※ 추출적 요약은 원문만을 사용하므로, 추상적 요약과 별도로 비교할 수 있습니다.

In [ ]:
from summa.summarizer import summarize

# 테스트 데이터의 첫 번째 기사를 대상으로 추출적 요약 수행
sample_article = encoder_input_test[0]
print('원문:', sample_article)
extractive_summary = summarize(sample_article)
print('추출적 요약:', extractive_summary if extractive_summary else '요약 결과가 충분하지 않습니다.')


## 최종 평가: 프로젝트 평가 루브릭 체크

아래 항목들을 통해 프로젝트의 각 단계가 평가 루브릭에 부합하는지 확인합니다.

[] 1. **텍스트 전처리 단계**:
   - 중복 및 Null 제거, 정규화, 불용어 제거, 데이터셋 분리, 정수 인코딩까지 체계적으로 진행됨.

[] 2. **텍스트 요약 모델 학습**:
   - 모델 학습 과정에서 train/validation loss의 감소 추세를 확인할 수 있으며,
     생성된 요약문에 핵심 단어들이 포함됨.

[] 3. **추출적 요약과 추상적 요약 비교**:
   - Summa를 이용한 추출적 요약과 seq2seq 기반 추상적 요약 결과를 비교 분석할 수 있음.

